#### Summary:
In this notebook we will read in a Seurat object with 10x multiome data, assign each barcode to a celltype and then run the SMORES method by cell type to link cREs to putative target genes. This Seurat object should consist of your final combined clusters with the cell types defined in a metadata column. Also it's recommended to only run SMORES on groups of cells (cell types) with 1,000 cells or more.

Required inputs:
- Per cell type bed file of all accessible peaks
- Per cell type text file of all expressed genes (we use pseudobulk TPM>1 for this)
- 

In [1]:
suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))

suppressMessages(library(dplyr))
suppressMessages(library(plyr))
suppressMessages(library(ggplot2))

suppressMessages(library(Matrix))
suppressMessages(library(matrixStats))
suppressMessages(library(MASS))
suppressMessages(library(data.table))
suppressMessages(library(stringr))

suppressMessages(library(qvalue))
suppressMessages(library(parallel))
suppressMessages(library(bettermc))

suppressMessages(library(tictoc))

In [2]:
set.seed(8)

# 1. Read in adata and prepare SMORES inputs

### Read in reference files

In [19]:
# Read in all celltype_peak files into a list

ct_peaks_list = list()

for (celltype in unique_cell_types){
    peak.fp <- sprintf('/path/to/celltype/accessible/peaks/%s.merged_peaks.anno.mergedOverlap.bed',celltype)
    ct.peaks <- read.table(peak.fp,sep='\t')
    ct_peaks <- paste(ct.peaks$V1,ct.peaks$V2,ct.peaks$V3,sep="_")
    ct_peaks_list[[celltype]] = ct_peaks
}

In [20]:
# Read in the gene coords ref file
ref_df = read.table('non-diabetic-islet-multiomics/references/gene_coords.gencodev32.hg38.bed', sep='\t', header=FALSE) #read in gene coords ref

### Read in the cell type labeled adata object

In [12]:
# Read in the adata object 
indir = "/dir/with/final/seurat/object"
rds_fp = file.path(indir,"final_object.rds")
tic()
adata = readRDS(rds_fp)
toc()
adata

elapsed time is 489.557000 seconds 


An object of class Seurat 
543177 features across 174819 samples within 4 assays 
Active assay: ATAC (210485 features, 210485 variable features)
 3 other assays present: RNA, SCT, ATAC_CTpeaks
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

In [13]:
# Assign celltypes to Idents (just take from metadata col! ours is called major_celltype)
Idents(adata) <- adata@meta.data$major_celltype

In [14]:
# Pull out list of unique cell types -- modify this to be the cell types you want to run SMORES on
cell_types <- levels(Idents(adata))
unique_cell_types <- unique(cell_types)

print("All cell types we can perform analyses on:")
print(cat(unique_cell_types,sep=", ")) #ignore NULL at the end

[1] "All cell types we can perform analyses on:"
beta, alpha, acinar, delta, gamma, immune, ductal, stellate, endothelial, schwannNULL


### Extract RNA counts

In [15]:
# Pull out rna counts obj
DefaultAssay(adata) <- 'RNA'
rna.counts <- GetAssayData(adata,slot='counts')
dim(rna.counts)

# Pull out sequencing depth vector
seq_depth_rna <- colSums(rna.counts)

# Read in gene coords ref file here just in case
ref_df <- read.table('non-diabetic-islet-multiomics/references/gene_coords.gencodev32.hg38.bed', sep='\t', header=FALSE) #read in gene coords ref

[1]  36601 174819

### Extract and binarize ATAC counts, set up peaks reference

In [16]:
# Extract and binarize the ATAC counts data
DefaultAssay(adata) <- 'ATAC_CTpeaks'
atac.counts <- GetAssayData(adata,slot='counts')
atac.counts.bin <- BinarizeCounts(atac.counts)

In [17]:
# Get data frame of all peaks in ATAC matrix
make_peaks_mtx <- function(atac.counts.bin){
    #extract peaks and modify naming convention
    peaks <- data.frame(names=row.names(atac.counts.bin))
    peaks$names <- gsub("-","_",peaks$names)
    
    #extract chr as seqnames
    getchr <- function(x){ return(strsplit(x,'_')[[1]][1])}
    peaks$seqnames <- lapply(peaks$names, getchr)
    peaks$seqnames <- as.character(peaks$seqnames)
    
    #extract start position
    getstart <- function(x){ return(strsplit(x,'_')[[1]][2])}
    peaks$start <- lapply(peaks$names, getstart)
    peaks$start <- as.numeric(peaks$start)
    
    #extract end position
    getend <- function(x){ return(strsplit(x,'_')[[1]][3])}
    peaks$end <- lapply(peaks$names, getend)
    peaks$end <- as.numeric(peaks$end)

    peaks <- data.table(peaks)
    peaks <- peaks[complete.cases(peaks),]
    return(peaks)
}

In [18]:
# Remake the peak names with _ instead of -
peak_names <- gsub('-','_',row.names(atac.counts.bin))
row.names(atac.counts.bin) <- peak_names

# Create the overall peaks matrix (position info for all peaks)
peaks <- make_peaks_mtx(atac.counts.bin)
total_npeaks <- nrow(peaks)

### Quick check that all data structures agree with each other
- Are the barcodes the same across RNA and ATAC objects?
- Do the ct_peaks match the ATAC object peaks?

In [21]:
table(colnames(atac.counts.bin) == colnames(rna.counts))
table(ct_peaks_list[['beta']] %in% row.names(atac.counts.bin))
table(ct_peaks_list[['gamma']] %in% row.names(atac.counts.bin))

### looks good!


  TRUE 
174819 


  TRUE 
112687 


  TRUE 
109718 

# 2. SMORES Functions

### Celltype and gene specific data processing

In [6]:
### Make a genomic range around a given gene using a coords file
get_ir_fin <- function(gene, ref_df, b_away=1000000){
    if (gene %in% ref_df$V4 == TRUE){
        ref_df_cut <- ref_df[ref_df$V4 ==gene,]
        chr        <- as.character(ref_df_cut$V1)
        if (ref_df_cut$V6 == '-'){
            tss <- max(c(ref_df_cut$V2,ref_df_cut$V3))
        } else {
            tss <- min((c(ref_df_cut$V2,ref_df_cut$V3)))
        }
        start  <- tss - b_away
        end    <- tss + b_away
        ir     <- IRanges(start = start, end = end, seqnames=chr)
        ir     <- data.table(data.frame(ir))
        return(ir)
    } else {
        ir <- IRanges(start = 0, end = 1, seqnames=NA)
        ir <- data.table(data.frame(ir))
        return(ir)
    }
}
    
### Find overlaps between ir (IRanges obj for one range) and the peaks list
get_overlaps <- function(ir, peaks){
    setkey(ir, seqnames, start, end)
    setkey(peaks, seqnames, start, end)
    overlaps <- foverlaps(peaks, ir, type="any")
    overlaps <- overlaps[complete.cases(overlaps),]
    return(overlaps)
}

### Create a matrix of peaks in the vicinity of a gene (from overlaps) that were called in
### a specific cell type (list is an input)
### Returns a sparse matrix of binarized ATAC counts for peaks active in a cell type overlapping a gene of interest
get_celltype_peaks_for_gene <- function(overlaps, atac.counts.bin, celltype.bcs, celltype, ct_peaks_list){
    overlapping_peaks <- overlaps$names
    
    # Read in peaks for celltype and use to filter overlapping_peaks    
    ct_peaks <- ct_peaks_list[[celltype]]    
    overlapping_peaks_ct <- overlapping_peaks[overlapping_peaks %in% ct_peaks]

    # Cut down atac df to filtered overlapping peaks
    # Check if there even are any overlapping peaks for this celltype
    if (length(overlapping_peaks_ct) < 1){
        fin_peaks_for_gene <- -1
    # If there's only one overlapping peak, gotta do a bit of a workaround...
    } else if (length(overlapping_peaks_ct) == 1){
        fin_peaks_for_gene <- as.data.frame(atac.counts.bin[overlapping_peaks_ct, celltype.bcs])
        colnames(fin_peaks_for_gene) <- overlapping_peaks_ct
    } else {
        peaks_for_gene <- atac.counts.bin[overlapping_peaks_ct,celltype.bcs]
        fin_peaks_for_gene <- t(as.data.frame(peaks_for_gene)) #transpose so that BCs are rownames and peaks are cols
    }
    return(fin_peaks_for_gene)    
}

### Normalization and Scaling

In [7]:
### Function to normalize gex for seq_depth 
### This returns the counts of the gene/million (similar to TPM but not normalized by gene length)
norm_gex <- function(gex, seq_depth){
    return(gex/(seq_depth/1e6))
}

### Actually running the method (with background permutations)

In [8]:
### Take the gex x cells and peaks x cells matrices and shuffle cell labels for both
### shuffles within sample using data.table
shuffle_BCs_vect <- function(y){
    dt <- data.table(sample=substr(y,1,4), barcode=y)
    dt <- setDT(dt)[, barcode2:=sample(barcode), by=sample]
    return(dt$barcode2)
}

### This is the function to parallelize and call for each iteration of permuting BCs
### It permutes the barcodes and the performs correlation between all peaks and the normalized gene expression
perform_cor_permuted_BCs <- function(iteration, peaks, gex){
    # Shuffle BCs for both objects within sample (peaks is a matrix, gex is a vector)
    peaks_perm <- peaks
    rownames(peaks_perm) <- shuffle_BCs_vect(rownames(peaks_perm))
    peaks_perm <- peaks_perm[rownames(peaks),]
    
    # If there's only one peak gotta do some extra df manipulation
    if (dim(peaks)[2] == 1){
        peaks_perm <- as.data.frame(peaks_perm)
        colnames(peaks_perm) <- colnames(peaks)
    }
    
    gex_perm <- gex
    new_bcs <- shuffle_BCs_vect(names(gex))
    names(gex_perm) <- new_bcs
    gex_perm <- gex_perm[names(gex)]
    
    # Perform correlation
    peak_cors <- apply(peaks_perm, 2, cor, x=gex_perm, method="spearman")
    return (peak_cors)
}

In [9]:
### This function compares the real correlation values to the shuffled background values
### To generate an empirical 2-sided pvalue for a correlation which it returns
### This assumes the REAL value is the first value in the distrib vector
get_cor_pvalue <- function(distrib, N){
    distrib <- abs(distrib)
    value <- distrib[1]
    background <- distrib[-1]
    
    # Compare value to ordered background to see significance
    pvalue <- length(background[background > value])/N
    return(pvalue)    
}

### Function to write the pvalues of ALL connections to a celltype specific bedpe file
extract_peak_pvalues <- function(cors, pvalues, peaks, ir, outdir, celltype, gene, b_away=1000000){
    # Get peak coords from pvalues names and gene coords from IR
    peak.info <- as.data.frame(str_split_fixed(peaks, '_', 3))
    gene.info <- ir[,c(4,1,2)]
    gene.info <- rbind(gene.info, gene.info[rep(1,length(pvalues)-1),])
    tss <- gene.info$start + b_away
    gene.info$start <- tss
    gene.info$end <- tss + 1
    bedpe.info <- cbind(peak.info, gene.info, as.data.frame(rep(gene,length(pvalues))), 
                       as.data.frame(cors), as.data.frame(pvalues))
    out_fp <- file.path(outdir, sprintf('%s_gene_tables',celltype), sprintf('%s_ALL_gene-peaks_corr_pvalues.bedpe', gene))
    write.table(bedpe.info, out_fp, sep='\t', col.names=FALSE, row.names=FALSE, quote=FALSE)
}

In [10]:
### This function organizes all the steps for each celltype-gene pair to correlate nearby cell type links
### with the gene's normalized expression and calculate an empirical p-value based on permutations
perform_cor_permuted_BCs_wrapper <- function(gene, celltype, adata, peaks, ct_peaks_list, ref_df, outdir, N){
    log_file = file.path(outdir, sprintf('%s_log.txt',celltype))
    #need to make SURE gene is in ref_df (otherwise get_ir_gin fails)
    if (!gene %in% ref_df$V4){
        write(sprintf('%s not in ref_df, cannot perform analysis %s', gene, Sys.time()), file=log_file, append=TRUE)
    } else {
        # Run basic analysis on correct BCs first
        # Read in gene expression data and normalize by seq_depth
        celltype.bcs <- names(Idents(adata)[Idents(adata) == celltype])
        gex_counts <- rna.counts[gene,celltype.bcs] #pull out counts for gene for celltype
        gex_counts_norm <- norm_gex(gex_counts, seq_depth_rna[celltype.bcs]) #normalize relevant counts by seq_depth

        # Set up necessary data structures for peaks (to correlate with gex)
        ir <- get_ir_fin(gene, ref_df) #get genomic range of gene
        overlaps <- get_overlaps(ir, peaks) #create a list of nearby peaks
        ctpeaks_for_gene <- get_celltype_peaks_for_gene(overlaps, atac.counts.bin, celltype.bcs, celltype, ct_peaks_list)

        # Quick check if we were able to get any ct peaks nearby the gene
        if (ctpeaks_for_gene == -1){
            write(sprintf('%s doesn\'t have any nearby peaks called in %s cells, cannot perform analysis %s', gene, celltype, Sys.time()), file=log_file, append=TRUE)
        } else {
            write(paste(gene, Sys.time()), file=log_file, append=TRUE)
            # Perform correlation between normalized gex counts and the penalized peaks
            real_peak_cors <- apply(ctpeaks_for_gene, 2, cor, x=gex_counts_norm, method="spearman")

            ### For N times, shuffle barcodes (WITHIN EACH SAMPLE) and repeat ^ steps to get underlying distribution
            cor_distribs <- lapply(seq(1:N),
                                     perform_cor_permuted_BCs, ctpeaks_for_gene, gex_counts_norm)

            # Convert background correlation runs into a data frame (rows are peaks, cols are correlation coefs)
            # Add in the real results as the first column
            all_cor_results <- data.frame(real_peak_cors)
            all_cor_results <- cbind(all_cor_results, as.data.frame(cor_distribs))
            colnames(all_cor_results) <- seq(0,N,1)    

            # Compare the real_peak_cors to the corresponding cor_distribs to calculate significance
            # Calculate empirical p-values based on assumption that correlation is positive/negative
            pvalues <- unlist(apply(all_cor_results, 1, get_cor_pvalue, N))

            # Write peak, gene coords, gene name and pvalue to a celltype file
            extract_peak_pvalues(real_peak_cors, pvalues, colnames(ctpeaks_for_gene), ir, outdir, celltype, gene)
        }   
    }
}


### Function which reads in the output file from a round of correlation permutation cRE-gene links for multiple genes
### and one celltype. It will then FDR correct all pvalues together, and output results to one file
fdr_correct_celltype_links <- function(celltype, genes, outdir, fdr=0.1){
    # Read in data into a list and then rbind
    gene_dfs <- list()
    missing_genes <- c()
    for (gene in genes){
        fp <- file.path(outdir, sprintf('%s_gene_tables',celltype), sprintf('%s_ALL_gene-peaks_corr_pvalues.bedpe', gene))
        if (file.exists(fp) == TRUE){
            gene_dfs[[gene]] <- read.table(fp, sep='\t', header=FALSE)
        } else {
            print(paste(gene,'not found', sep=' '))
            missing_genes <- c(missing_genes, gene)
        }  
    }
    # Write out missing genes to a file? ### maybe add this in later if I want TO DO
    
    # Make one df with all results and save to file
    df <- rbindlist(gene_dfs)
    colnames(df) <- c('chr1','start1','end1','chr2','start2','end2','gene','corr_coef','pvalue')
    out_fp1 <- file.path(outdir, sprintf('%s_ALL_ct_peaks_corr_pvalues.bedpe', celltype))
    write.table(df, out_fp1, sep='\t', col.names=FALSE, row.names=FALSE, quote=FALSE)
    
    # Calculate qvalues with Storey's method, then add to the df and write a new filtered file (based on fdr threshold)
    qvalues <- try(suppressWarnings(qvalue(df$pvalue, fdr.level=fdr)))
    df$qvalue <- qvalues$qvalue
    signif_df <- df[df$qvalue < fdr,]
    return(signif_df)
}

In [11]:
### Function which loops through a list of celltypes and genes and runs the correlation + permutation method
### on all pairs of them (every gene for every celltype)
### After looping through all genes, FDR corrects ALL peak p-values for a celltype together
perform_ct_gene_cor_wrapper <- function(adata, peaks, ct_peaks_list, celltypes, genes, outdir, num_genes_parallel){
    fdr <- 0.1
    num_cores <- num_genes_parallel
    
    for (celltype in celltypes){
        dir.create(sprintf('%s/%s_gene_tables',outdir,celltype), showWarnings=FALSE)
        bettermc::mclapply(genes, perform_cor_permuted_BCs_wrapper, celltype, adata, peaks, 
                   ct_peaks_list, ref_df, outdir, N=100,
                   mc.cores = num_cores, mc.preschedule=FALSE)
        
        # FDR correct pvalues together and write output file
        signif_df <- fdr_correct_celltype_links(celltype, genes, outdir, fdr)
        out_fp <- file.path(outdir, sprintf('%s_linked_ct_peaks_fdr%s_corr.bedpe',celltype, fdr))
        write.table(signif_df, out_fp, sep='\t', col.names=FALSE, row.names=FALSE, quote=FALSE)                

        # Delete all individual gene files (whole dir)
        gene_dir <- file.path(outdir, sprintf('%s_gene_tables', celltype))
        system(sprintf('rm -r %s', gene_dir))
    }
}

# 3. Run SMORES method

In [22]:
### Function for running SMORES on an individual cell type (gets genes list, etc)
run_link_corr_ct <- function(celltype, genes_dir, outdir, log_fp){
    # Write time to overall log file
    write(sprintf('Starting cRE-gene correlation for %s cells: %s', celltype, Sys.time()), file=log_fp, append=TRUE)

    # Make a subdir of outdir for this celltype
    ct_outdir <- file.path(outdir, celltype)
    print(ct_outdir)
    dir.create(ct_outdir, showWarnings=FALSE)
    
    # Get all genes expressed in the celltype
    fp <- file.path(genes_dir, sprintf('%s_expressed_genes_TPM1.txt', celltype))
    ct_genes <- scan(fp, what='', sep='\n')
    ct_genes <- ct_genes[ct_genes %in% ref_df$V4]

    # Remove any genes with '/' in the name
    ct_genes <- ct_genes[!grepl('/',ct_genes)]
    write(sprintf('Number of genes expressed in %s cells: %s', celltype, length(ct_genes)), file=log_fp, append=TRUE)
    
    # Get the number of cells within this celltype (just for fun)
    num_cells <- length(Idents(adata)[Idents(adata) == celltype])
    write(sprintf('Number of cells classified as %s cells: %s', celltype, num_cells), file=log_fp, append=TRUE)
    
    # Run the correlation-based links method for all genes in parallel!
    perform_ct_gene_cor_wrapper(adata, peaks, ct_peaks_list, c(celltype), ct_genes, ct_outdir, 10)
    
    # Write time to overall log file
    write(sprintf('Finished cRE-gene correlation for %s cells: %s', celltype, Sys.time()), file=log_fp, append=TRUE)
    write(' ', file=log_fp, append=TRUE)
}

In [ ]:
### Ex 1: One cell type SMORES run (beta cells)

ct <- 'beta'
genes_dir <- '/path/to/expressed/gene/lists'
outdir <- '/path/to/write/outputs/to'
log_fp <- file.path(outdir, 'all_genes_log.txt')

run_link_corr_ct(ct, genes_dir, outdir, log_fp)

In [ ]:
### Ex 2: Run SMORES on multiple cell types (one after the other)
for (ct in celltypes[-1]){
    genes_dir <- '/path/to/expressed/gene/lists'
    outdir <- '/path/to/write/outputs/to'
    log_fp <- file.path(outdir, 'all_genes_log.txt')

    run_link_corr_ct(ct, genes_dir, outdir, log_fp)
}

In [2]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.9.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.9.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] tictoc_1.1         bettermc_1.1.2     qvalue_2.26.0      stringr_1.4.1     
 [5] data.table_1.14.8  MASS_7.3-58.1      matrixStats_1.3.0  Matrix_1.5-1      
 [9] ggplot2_3.4.4      plyr_1.8.9         dplyr_1.0.10       sp_1.5-1          
[13] Seurat